In [1]:
import os
import pandas as pd
import scanpy as sc

from arboreto.algo import grnboost2, genie3
from arboreto.utils import load_tf_names
from distributed import Client, LocalCluster
# WORKED!!!!!! 

In [2]:
# read in expression matrix 
# set a working directory
wdir = "/lustre/groups/ml01/workspace/samantha.bening/Bachelor/"
os.chdir( wdir )

adata = sc.read_h5ad('data/adata.h5ad')
adata

AnnData object with n_obs × n_vars = 5397 × 16719
    obs: 'sampleID', 'barcode', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_20_genes', 'total_counts_mt', 'log1p_total_counts_mt', 'pct_counts_mt', 'total_counts_ribo', 'log1p_total_counts_ribo', 'pct_counts_ribo', 'total_counts_hb', 'log1p_total_counts_hb', 'pct_counts_hb', 'outlier', 'mt_outlier', '_scvi_batch', '_scvi_labels', 'leiden_res0_6', 'manual_celltype_annotation', 'celltypist_cell_label', 'celltypist_conf_score', 'celltypist_cell_label_coarse'
    var: 'n_cells', 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: '_scvi_manager_uuid', '_scvi_uuid', 'celltypist_cell_label_coarse_colors', 'celltypist_cell_label_colors', 'hvg', 'leiden', 'leiden_res0_6_colors', 'neighbors', 'pca', 'sampleID_colors', 'umap'
    obsm: 'X_pca', 'X_scVI', 'X_umap'
    varm: 'PCs'
    layers: 'int_norm', 'log_int_norm', 'log_norm', 'norm', 'raw'
    obsp: 'connectivitie

In [3]:
# subset for a very broad cell type to test 

adata = adata[adata.obs['celltypist_cell_label_coarse'] == 'Plasma']
# filter for genes not expressed in e.g. 30 or more cells
sc.pp.filter_genes(adata, min_cells=30)
# make expression matrix 
ex_matrix = adata.to_df(layer='log_norm')
ex_matrix

/home/icb/samantha.bening/tools/apps/mamba/envs/grnboost/lib/python3.12/site-packages/scanpy/preprocessing/_simple.py:250: ImplicitModificationWarning: Trying to modify attribute `.var` of view, initializing view as actual.
  adata.var['n_cells'] = number


index,AL669831.5,LINC00115,NOC2L,PLEKHN1,HES4,ISG15,TNFRSF18,TNFRSF4,SDF4,B3GALT6,...,MT-CO3,MT-ND3,MT-ND4L,MT-ND4,MT-ND5,MT-ND6,MT-CYB,AL592183.1,AC233755.2,AC233755.1
CID005217-1-AAGTGAACATTCAGGT,0.89861,0.0,0.547019,0.0,0.000000,1.534815,0.000000,0.000000,0.898610,0.000000,...,4.598071,3.380846,1.534815,3.964513,3.160012,0.547019,4.152865,0.0,0.0,0.0
CID005217-1-ACGTAACTCGTGAGAG,0.00000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,4.553841,3.921417,0.000000,4.298085,2.652714,0.974206,4.072733,0.0,0.0,0.0
CID005217-1-ACTGTGACAAAGCAAT,0.00000,0.0,0.000000,0.0,0.882427,1.658280,0.882427,0.000000,0.000000,0.000000,...,4.751497,4.254494,2.890433,4.606899,3.772824,0.882427,4.486826,0.0,0.0,0.0
CID005217-1-ATTCAGGGTCTGATCA,0.00000,0.0,0.000000,0.0,1.698778,1.470279,0.000000,0.000000,0.000000,0.749913,...,4.385694,4.283258,2.176762,4.115911,3.438943,0.749913,4.478604,0.0,0.0,0.0
CID005217-1-CAAGACTTCTAAGGAA,0.00000,0.0,0.000000,0.0,0.000000,0.970312,0.000000,0.000000,0.970312,0.000000,...,3.589985,2.703341,1.453377,3.812131,3.241870,0.598498,3.736936,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
CID008474-1-TTTCATGAGGACAGCT,0.00000,0.0,0.000000,0.0,0.000000,0.000000,0.495698,0.000000,0.000000,0.000000,...,3.439047,2.797248,0.000000,2.835628,1.073271,0.000000,3.259023,0.0,0.0,0.0
CID008474-1-TTTCCTCGTTTGGAGG,0.00000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,3.602093,3.323458,0.821586,2.935847,2.523080,0.000000,3.368339,0.0,0.0,0.0
CID008474-1-TTTCGATTCTGCAGCG,0.00000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.410058,0.000000,...,2.091454,1.396600,0.000000,1.715985,1.396600,0.000000,2.538355,0.0,0.0,0.0
CID008474-1-TTTGATCGTGCCGGTT,0.00000,0.0,0.684229,0.0,0.000000,0.000000,0.684229,0.684229,0.000000,0.000000,...,2.816359,2.548422,0.684229,2.755801,1.086703,0.000000,3.073947,0.0,0.0,0.0


In [4]:
# make 'tf_list' which in this case is just the genes themselves (for gene-gene adjacencies)

tf_names = list(adata.var_names)
len(tf_names)

9955

In [ ]:
# select specific cell type (four or five v coars cell types)
# may be different per cell, can show all these networks side by side
# filter out very lowely expressed genes

## Create custom Dask Client

In [5]:
local_cluster = LocalCluster(n_workers=18, 
                             threads_per_worker=20) # put in the number of cores you gave the job

custom_client = Client(local_cluster)
custom_client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 18
Total threads: 360,Total memory: 200.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:44627,Workers: 18
Dashboard: http://127.0.0.1:8787/status,Total threads: 360
Started: Just now,Total memory: 200.00 GiB
Comm: tcp://127.0.0.1:42275,Total threads: 20
Dashboard: http://127.0.0.1:41109/status,Memory: 11.11 GiB
Nanny: tcp://127.0.0.1:41771,


## Run GRNBoost2

In [6]:
%%time
network = grnboost2(expression_data=ex_matrix,
                    tf_names=tf_names,
                    client_or_address=custom_client)

/home/icb/samantha.bening/tools/apps/mamba/envs/grnboost/lib/python3.12/site-packages/distributed/client.py:3169: UserWarning: Sending large graph of size 70.29 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(


2024-03-20 17:23:52,585 - tornado.application - ERROR - Uncaught exception GET /status/ws (10.233.0.250)
HTTPServerRequest(protocol='http', host='cpusrv05:8787', method='GET', uri='/status/ws', version='HTTP/1.1', remote_ip='10.233.0.250')
Traceback (most recent call last):
  File "/home/icb/samantha.bening/tools/apps/mamba/envs/grnboost/lib/python3.12/site-packages/tornado/web.py", line 1790, in _execute
    result = await result
             ^^^^^^^^^^^^
  File "/home/icb/samantha.bening/tools/apps/mamba/envs/grnboost/lib/python3.12/site-packages/tornado/websocket.py", line 273, in get
    await self.ws_connection.accept_connection(self)
  File "/home/icb/samantha.bening/tools/apps/mamba/envs/grnboost/lib/python3.12/site-packages/tornado/websocket.py", line 863, in accept_connection
    await self._accept_connection(handler)
  File "/home/icb/samantha.bening/tools/apps/mamba/envs/grnboost/lib/python3.12/site-packages/tornado/websocket.py", line 946, in _accept_connection
    await se

CPU times: user 1min 43s, sys: 24.9 s, total: 2min 7s
Wall time: 30min 49s


In [7]:
network.head()

,TF,target,importance
3186,HSPA1B,HSPA1A,25.276421
3187,HSPA1B,HSP90AA1,21.279992
3186,HSPA1A,HSPA1B,19.230483
9620,IGLC3,IGKC,19.111079
3186,HSPA1A,HSP90AA1,16.197288


In [9]:
network.to_csv('SCENICfiles/gene_gene_adj.csv',  header=False, index=False)